In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pyplot
np = numpy
plt = pyplot
from pylab import *
from numpy import *
from scipy.optimize import curve_fit
%matplotlib

Using matplotlib backend: TkAgg


In [32]:
import matplotlib as mpl
cmap = mpl.colormaps['viridis']
cmap_turbo = mpl.colormaps['turbo']

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [4]:
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

In [5]:
%run my_functions

# Load data file names
I load the file names of data. You have to write the data directory in `file_path` then the program sorts the file names into several categories, according to the their SimuType, and their nature.

### SimuType
- `Langevin` is the first initialisation of the cloud with the Langevin program.
- `SimuTypeQ` is the first part with velocity-Verlet. Introduces laser cooling and in a second time a ramp on the RF voltage.
- `SimuType4` is the second part. Introduces the Giant Molecule with the proper kinetic energy. Laser cooling and trapping parameters remains constant. Giant Molecule evacuated at the end of `SimuType4`.
- `SimuType2` is the third part. Laser cooling and trapping parameters remains constant.

### Nature of files
- `xva` contain a snapshot of the cloud : positions, velocities and accelerations of each ion at a given time, at the end of each SimuType, under the format `.bin`. For `SimuType4`, files with `xva` and `.dat` extension contain information about the Giant Molecule.
- `Temp` contain temperature of cloud as a function of time, under the form of a `.dat`.
- `trj` contain trajectory as a function of time : `trj` alone is for one or many Ca+ ions, `trjGMol` is the trajectory of the GMol.

In [60]:
# for i,j in enumerate(os.walk(file_path)):
#     print(i,j)
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20230331'+'/DC08_RF08_050eV'+'/Try00'
# print(os.listdir(file_path))

file_name_Lan = sort([x for x in os.listdir(file_path) if 'Langevin' in x or '3D' in x])
print(file_name_Lan)
file_name_SimuTypeQ = sort([x for x in os.listdir(file_path) if 'SimuTypeQ' in x])
print(file_name_SimuTypeQ)
file_name_SimuType4_xva = sort([x for x in os.listdir(file_path) if 'SimuType4' in x and 'xva' in x])
print(file_name_SimuType4_xva)
file_name_SimuType4_trjGMol = sort([x for x in os.listdir(file_path) if 'SimuType4' in x and 'trj' in x and 'GMol' in x])
print(file_name_SimuType4_trjGMol)
file_name_SimuType2_xva = sort([x for x in os.listdir(file_path) if 'SimuType2' in x and 'xva' in x])
print(file_name_SimuType2_xva)

file_name_SimuType4_Temp = sort([x for x in os.listdir(file_path) if 'SimuType4' in x and 'Temp' in x])
print(file_name_SimuType4_Temp)
file_name_SimuType2_Temp = sort([x for x in os.listdir(file_path) if 'SimuType2' in x and 'Temp' in x])
print(file_name_SimuType2_Temp)

['Langevin_cooling.info'
 'Temp_3D_Harmo_N01024_T***uK_F0.15D-19Kg_sRFG.bin'
 'xva_3D_Harmo_N01024_T***uK_F0.15D-19Kg_sRFG.bin']
['Temp_SimuTypeQ_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuTypeQ_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG.bin'
 'xva_SimuTypeQ_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG.info']
['xva_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_0.000E+00_50eV.bin'
 'xva_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_0.000E+00_50eV.dat'
 'xva_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_0.000E+00_50eV.info'
 'xva_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-04_50eV.bin'
 'xva_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-04_50eV.dat'
 'xva_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-04_50eV.info'
 'xva_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-05_50eV.bin'
 'xva_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1

# Reading snapshots

In [61]:
x0 = 10e-6 # [0, 1e-6, 5e-6, 10e-6, 50e-6, 100e-6]
str_x0 = f'x0_{x0:.3E}'
print(str_x0)
xva_Langevin  = load_xva_init_bin_Lan(file_path+'/'+sort(file_name_Lan)[2].strip('.bin'),1)[5]
xva_SimuTypeQ = load_xyz_init_bin_DP(file_path+'/'+sort(file_name_SimuTypeQ)[1].strip('.bin'))[0]
xva_SimuType4 = load_xyz_init_bin_DP(file_path+'/'+sort([x for x in file_name_SimuType4_xva if str_x0 in x])[0].strip('.bin'))[0]
xva_SimuType2 = load_xyz_init_bin_DP(file_path+'/'+sort([x for x in file_name_SimuType2_xva if str_x0 in x])[0].strip('.bin'))[0]

x0_1.000E-05


In [62]:
to_plot = xva_SimuType4
figure('xva'); clf()
# title(fig_title)
subplot(211,aspect=1.0)
plot(to_plot [0,:]*1e3,to_plot [1,:]*1e3,'8',color='xkcd:purplish blue')
xlabel('x[µm]')
ylabel('y[µm]')
grid()
# xlim(-1000,1000)
# ylim(-1000,1000)

# subplot(212,aspect=1.0)
subplot(212)
plot(to_plot [2,:]*1e3,to_plot [0,:]*1e3,'8',color='xkcd:purplish blue')
xlabel('z[µm]')
ylabel('x[µm]')
grid()
# xlim(-1000,1000)
# ylim(-1000,1000)

tight_layout()
# savefig('xva_4.png',dpi=300)

In [10]:
max(to_plot [2,:]*1e3) - min(to_plot [2,:]*1e3)

470.3225756780485

# Reading temperature

In [63]:
Temp_Langevin  = load_Temp_init_bin_Lan(file_path+'/'+sort(file_name_Lan)[1].strip('.bin'),1)[5]
Temp_SimuTypeQ = load_T_and_PM_simu(file_path+'/'+sort(file_name_SimuTypeQ)[0].strip('.dat'))
Temp_SimuType4 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType4_Temp if str_x0 in x])[0].strip('.dat'))
Temp_SimuType2 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType2_Temp if str_x0 in x])[0].strip('.dat'))
print(sort(file_name_SimuTypeQ)[0])

[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.001]
[5.05000000e-07 4.75582168e-07 2.90796629e-07 6.50697151e-07
 1.14340956e+02 3.85980177e+01 1.86685223e+02]
Temp_SimuTypeQ_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG.dat


In [64]:
figure('Temperature_Lan',clear='True')
# subplot(211)
semilogy(Temp_Langevin[:,0]*1e3,mean(Temp_Langevin[:,4:7],axis=1))
grid()
xlabel('t [ms]')
ylabel('T [K]')
tight_layout()

In [65]:
figure('Temperature',clear='True')
# subplot(211)
semilogy(Temp_Langevin[:,0]*1e3,mean(Temp_Langevin[:,4:7],axis=1))
semilogy(Temp_SimuTypeQ[0]*1e3,mean(Temp_SimuTypeQ[2],axis=1))
semilogy(Temp_SimuType4[0]*1e3,mean(Temp_SimuType4[2],axis=1))
semilogy(Temp_SimuType2[0]*1e3,mean(Temp_SimuType2[2],axis=1))
grid()
xlabel('t [ms]')
ylabel('T [K]')
tight_layout()

In [67]:
# En boucle

x0_list = [0, 1e-6, 5e-6, 10e-6, 50e-6, 75e-6, 100e-6]
my_dash = ['-','-','-','-','-','-','-']

figure('Temperature_loop',clear='True')

Temp_Langevin  = load_Temp_init_bin_Lan(file_path+'/'+sort(file_name_Lan)[1].strip('.bin'),1)[5]
Temp_SimuTypeQ = load_T_and_PM_simu(file_path+'/'+sort(file_name_SimuTypeQ)[0].strip('.dat'))
semilogy(Temp_Langevin[:,0]*1e3,mean(Temp_Langevin[:,4:7],axis=1))
semilogy(Temp_SimuTypeQ[0]*1e3,mean(Temp_SimuTypeQ[2],axis=1))

for i,j in enumerate(x0_list):
    str_x0 = f'{j:.3E}'
    print(i,str_x0)
    Temp_SimuType4 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType4_Temp if str_x0 in x])[0].strip('.dat'))
    Temp_SimuType2 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType2_Temp if str_x0 in x])[0].strip('.dat'))
    # subplot(211)

    semilogy(Temp_SimuType4[0]*1e3,mean(Temp_SimuType4[2],axis=1),color='C2',ls=my_dash[i])
    semilogy(Temp_SimuType2[0]*1e3,mean(Temp_SimuType2[2],axis=1),color=cmap_turbo(((i)/6)),ls=my_dash[i],label=x0_list[i]*1e6)
grid()
legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
xlabel('t [ms]')
ylabel('T [K]')
xlim(8.35,9.2)
# ylim(2e-4,2e2)
tight_layout()
savefig('x0var_zoom_50eV.png',dpi=300)

[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.001]
[5.05000000e-07 4.75582168e-07 2.90796629e-07 6.50697151e-07
 1.14340956e+02 3.85980177e+01 1.86685223e+02]
0 0.000E+00
1 1.000E-06
2 5.000E-06
3 1.000E-05
4 5.000E-05
5 7.500E-05
6 1.000E-04


# Reading GMol trajectory

In [68]:
def load_trj_GMol(str_load):
    fid = open(str_load+'.bin', 'rb')
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    j_save_temp  = fromfile(fid, int32,1);
    junk = fromfile(fid, int32,1)        # Read record stop tag

    junk = fromfile(fid, int32,1)        # Read record start tag
    steps  = fromfile(fid, int32,1);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    print(junk)
    jmax  = fromfile(fid, int32,1);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)
    
    junk = fromfile(fid, int32,1)        # Read record start 
    print(junk)
    t_act_gmol  = fromfile(fid, float64, jmax[0]+2);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    print(junk)
    trj_gmol  = fromfile(fid, float64, (jmax[0]+2)*6);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    print(junk)
    Ep_gmol  = fromfile(fid, float64, jmax[0]+2);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)

    fid.close

    return j_save_temp, steps, jmax, t_act_gmol, trj_gmol, Ep_gmol

In [69]:
for i,j in enumerate(x0_list):
#     file_name_SimuType4_trjGMol
    str_x0 = f'{j:.3E}'
    tmp = [x for x in file_name_SimuType4_trjGMol if str_x0 in x][0].strip('.bin')
    print(tmp)

trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_0.000E+00_50eV_GMol
trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-06_50eV_GMol
trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_5.000E-06_50eV_GMol
trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-05_50eV_GMol
trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_5.000E-05_50eV_GMol
trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_7.500E-05_50eV_GMol
trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-04_50eV_GMol


In [70]:
time_gmol = []
trj_gmol = []
steps_gmol = []

for i,j in enumerate(x0_list):
#     file_name_SimuType4_trjGMol
    str_x0 = f'{j:.3E}'
    tmp = [x for x in file_name_SimuType4_trjGMol if str_x0 in x][0].strip('.bin')
    str_load = f'{file_path}/{tmp}'

    print(str_load)
    # steps, t_act_gmol, Ep_gmol, trj_gmol = load_trj_GMol(str_load)
    j_save_temp, steps, jmax, t_act_gmol, trj_gmol_tmp, Ep_gmol = load_trj_GMol(str_load)
    trj_gmol.append( reshape(trj_gmol_tmp,(6,(jmax[0]+2)),order='F') )
    time_gmol.append( t_act_gmol )
    steps_gmol.append( steps[0] )

/home/adrien/RemoteFS/Rivendel/Simulations/20230331/DC08_RF08_050eV/Try00/trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_0.000E+00_50eV_GMol
[4]
[4]
[800816]
[800816]
[4804896]
[4804896]
[800816]
[800816]
/home/adrien/RemoteFS/Rivendel/Simulations/20230331/DC08_RF08_050eV/Try00/trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-06_50eV_GMol
[4]
[4]
[800816]
[800816]
[4804896]
[4804896]
[800816]
[800816]
/home/adrien/RemoteFS/Rivendel/Simulations/20230331/DC08_RF08_050eV/Try00/trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_5.000E-06_50eV_GMol
[4]
[4]
[800816]
[800816]
[4804896]
[4804896]
[800816]
[800816]
/home/adrien/RemoteFS/Rivendel/Simulations/20230331/DC08_RF08_050eV/Try00/trj_SimuType4_01_N01024_Vrf0051_Udc0.2631D+01V_D1.0_S1.0RFG_Q10_x0_1.000E-05_50eV_GMol
[4]
[4]
[800816]
[800816]
[4804896]
[4804896]
[800816]
[800816]
/home/adrien/RemoteFS/Rivendel/Simulations/20230331/DC08_RF08_050eV/Try00/trj_SimuType4_01_N01024_Vr

In [74]:
figname = 'GMol_trajectory_xz'
fig = plt.figure(figname,clear='True')
# fig.set_size_inches(11.69,8.27)
ax = fig.add_subplot(111)
# subplot(211)
# to_plot = xva_SimuTypeQ
# ax.plot(to_plot[2,:]*1e3,to_plot[0,:]*1e6,'8',color='xkcd:purplish blue')

for k in range(len(file_name_SimuType4_trjGMol)):
    ax.plot(trj_gmol[k][2][:steps_gmol[k]]*1e6,trj_gmol[k][0][:steps_gmol[k]]*1e9-x0_list[k]*1e9, # -x0_list[k]*1e9
         '+',label=x0_list[k]*1e6,color=cmap_turbo(((k)/6)))
    
ax.grid()
ax.set_xlabel('z [µm]')
ax.set_ylabel('x-$x_0$ [nm]')
# xlim(-255,-181)
# ylim(-1500,17500)
ax.legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
tight_layout()
plt.savefig(figname+'_50eV.png',dpi=300)

In [75]:
figname = 'GMol_trajectory_yz'
fig = plt.figure(figname,clear='True')
fig.set_size_inches(11.69,8.27)
ax = fig.add_subplot(111)
for k in range(len(file_name_SimuType4_trjGMol)):
    ax.plot(trj_gmol[k][2][:steps_gmol[k]]*1e6,trj_gmol[k][1][:steps_gmol[k]]*1e9, # -x0_list[k]*1e9
         '+',label=x0_list[k]*1e6,color=cmap_turbo((k/6)))
ax.grid()
ax.set_xlabel('z [µm]')
ax.set_ylabel('y [nm]')
# xlim(-255,-181)
# ylim(-72,72)
ax.legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
tight_layout()
plt.savefig(figname+'_50eV.png',dpi=300)

In [43]:
figname = 'GMol_trajectory_xy'
fig = plt.figure(figname,clear='True')
fig.set_size_inches(11.69,8.27)
ax = fig.add_subplot(111)
for k in range(len(file_name_SimuType4_trjGMol)):
    ax.plot(trj_gmol[k][0][:steps_gmol[k]]*1e9,trj_gmol[k][1][:steps_gmol[k]]*1e9, # -x0_list[k]*1e9
         '+',label=x0_list[k]*1e6)
ax.grid()
ax.set_xlabel('x [nm]')
ax.set_ylabel('y [nm]')
# xlim(-255,-181)
# ylim(-72,72)
ax.legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
tight_layout()
# plt.savefig(figname+'.png',dpi=300)

In [81]:
figname = 'GMol_velocity_z'
fig = plt.figure(figname,clear='True')
fig.set_size_inches(11.69,8.27)
ax = fig.add_subplot(111)
# subplot(211)
for k in range(len(file_name_SimuType4_trjGMol)):
    ax.plot(trj_gmol[k][2][:steps_gmol[k]]*1e6,trj_gmol[k][5][:steps_gmol[k]],
         '',label=x0_list[k]*1e6,color=cmap_turbo((k/6))) # ,color=cmap(k/5)
ax.grid()
ax.set_xlabel('z [µm]')
ax.set_ylabel('$v_z$ [m/s]')
ax.legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
ax.set_xlim(-500,500)
ax.set_ylim(97.75,98.17)
tight_layout()
plt.savefig(figname+'_zoom_50eV.png',dpi=300)

In [82]:
m_GM = 1e6*1.66054e-27 # la masse de la GMol
C_e = 1.602e-19        # Coulomb

DeltaE = []
for k in range(len(file_name_SimuType4_trjGMol)):
    v_i = sqrt( trj_gmol[k][3][0]**2+trj_gmol[k][4][0]**2+trj_gmol[k][5][0]**2 )
    v_f = sqrt( trj_gmol[k][3][steps_gmol[k]-1]**2  +
               trj_gmol[k][4][steps_gmol[k]-1]**2 +
               trj_gmol[k][5][steps_gmol[k]-1]**2 )
    DeltaE.append( 0.5*m_GM*(v_f**2-v_i**2) / C_e )
print(DeltaE)

[-0.14797648352826043, -0.14070942180835963, -0.1331791481843398, -0.19278536121147755, -0.07924839002804082, -0.020438321572911294, -0.005080762342714484]


In [56]:
def my_lin_fit(x,a,b):
    return a*x+b    

In [84]:
# popt,pcov = curve_fit(my_lin_fit,
#                       x0_list,
#                       DeltaE)

figname = 'delta_E_vs_x0'
figure(figname,clear=True)
plot(multiply(x0_list,1e6),DeltaE,
     marker='P',ls='',ms=12,color='xkcd:rouge')
# plot(linspace(0,20*1e-6,50),my_lin_fit(linspace(0,20*1e-6,50),*popt),label='lin. fit')
text(22,-0.042,'Q=10,E=50 eV')
grid()
xlabel('$x_0$ [µm]')
ylabel('$\Delta E$ [eV]')
# legend()
tight_layout()
savefig(figname+'_50eV.png',dpi=300)